In this program, Google opening stock price for month January(2017) is predicted. Last 5 year's data of Google stock price is used for analysis. 



## Step 1 : Import Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

## Step 2: Import Data

In [ ]:
train = pd.read_csv('Google_Stock_Price_Train.csv')

In [ ]:
train.head()

In [ ]:
training_set = train.iloc[:,1:2].values

## Step 3: Split data into feature and label

In order to train the model, we need to provide it inputs as well as it's coreeseponding output. That's the reason why we split the data into feature (input) and label (output).

Transforms features by scaling each feature to a given range.

In [ ]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

This estimator scales and translates each feature individually such that it is in the given range on the training set

In [ ]:
X_train=[]
y_train=[]
for i in range(60,1258):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

reshape it to (batche_size(#size of inputs),timesteps,input_dimension)


In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train.shape, y_train.shape

## Step 4: Create and fit the LSTM network
Here we are applying LSTM on the training dataset.
LSTM networks are well-suited to classifying, processing and making predictions based on time series data, since there can be lags of unknown duration between important events in a time series. LSTMs deal with the exploding and vanishing gradient problems that can be encountered when training traditional RNNs.

In [ ]:
regressor = Sequential()

In [ ]:
regressor.add(LSTM(units=100,return_sequences=True,input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.4))

In [ ]:
regressor.add(LSTM(units=100,return_sequences=True))
regressor.add(Dropout(0.6))

In [ ]:
regressor.add(LSTM(units=100,return_sequences=True))
regressor.add(Dropout(0.6))

In [ ]:
regressor.add(LSTM(units=100))
regressor.add(Dropout(0.4))

In [ ]:
regressor.add(Dense(units=1))

Before training a model, you need to configure the learning process, which is done via the compile method

In [ ]:
regressor.compile(optimizer='adam',loss='mean_squared_error')

Fit the data onto RAM in the batches of size 32 and run it for 100 epochs

In [ ]:
regressor.fit(X_train,y_train,epochs=100,batch_size=32)

## Step 5: Testing 
In this step we'll provide our model unseen data and compute the right and wrong predictions. From this we can calculate the accuracy of our model. 

Read the data for testing.

In [ ]:
test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = test.iloc[:, 1:2].values
total_data=pd.concat([train['Open'],test['Open']],axis=0)

In LSTM we predict values based on past values and current input. Here we are predicting 246 values, using past 60 from the train data.

Now we'll reshape and sale the data

In [ ]:
inputs=total_data[len(train)-len(test)-60:].values
inputs=inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
X_test=[]
for i in range(60,80):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)
X_test.shape
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

## Predict the stock price 
inverse_transform --> Undo the scaling of X according to feature_range.

In [ ]:
predicted_stock_price=regressor.predict(X_test)
predicted_stock_price=sc.inverse_transform(predicted_stock_price)

## Graph showing Predicted Stock Price and Actual Stock Price 

In [ ]:
#Visualization
plt.rcParams["figure.figsize"] = (12,8)
plt.plot(predicted_stock_price,color='red',label='Predicted Stock Price')
plt.plot(real_stock_price,color='blue',label='Real Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
with open('model.pkl','wb') as file:
    pickle.dump(regression, file)